In [10]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
url = 'https://hacom.vn/laptop'
driver.get(url)

def get_product_links(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    links = soup.select('h3.p-name a')
    product_links = [f'https://hacom.vn{link["href"]}' for link in links] if links else None
    return product_links
all_product_links = []
# Lặp qua từng trang
for page_number in range(1, 15): 
    # Truy cập trang
    driver.get(url + f'/{page_number}/')

    # Chờ để đảm bảo trang đã được tải đủ
    time.sleep(2)

    # Lấy dữ liệu từ trang hiện tại
    current_page_source = driver.page_source
    current_product_links = get_product_links(current_page_source)

    # In ra các link sản phẩm từ trang hiện tại
    for link in current_product_links:
        all_product_links.append(link)
print(all_product_links)
# Đóng trình duyệt khi hoàn thành
driver.quit()

['https://hacom.vn/laptop-acer-aspire-5-a514-56p-55k5-nx.khrsv.003-i5-1335u-16gb-ram-512gb-ssd-14.0-inch-wuxga-ips-win11-xam-2023', 'https://hacom.vn/laptop-lenovo-ideapad-slim-5-14ilr8-82xd002vvn-i5-13500h-16gb-ram-512gb-ssd-14-wuxga-oled-win11-xam', 'https://hacom.vn/laptop-dell-inspiron-3520-71027003-i5-1235u-8gb-ram-512gb-ssd-15.6-inch-fhd-win11-officehs21-den', 'https://hacom.vn/laptop-dell-inspiron-3520-d5n53-i3-1115g4-8gb-ram-256gb-ssd-15.6-inch-fhd-win11-den-nk_bao-hanh-tai-hacom', 'https://hacom.vn/laptop-dell-inspiron-3520-25p231-i5-1235u-16gb-ram-512gb-ssd-15.6-inch-fhd-120hz-win11-officehs21-den', 'https://hacom.vn/laptop-asus-vivobook-tn3402ya-lz192w-r5-7530u-16gb-ram-512gb-ssd-14-cam-ung-win11-bac', 'https://hacom.vn/laptop-asus-vivobook-m1605ya-mb303w-r7-7730u-16gb-ram-512gb-ssd-16-wuxga-win11-bac', 'https://hacom.vn/laptop-asus-vivobook-x1504za-nj582w-i3-1215u-8gb-ram-512gb-ssd-15.6-fhd-win11-bac', 'https://hacom.vn/laptop-asus-vivobook-a1505za-l1245w', 'https://hacom.v

In [28]:
import requests
def get_product_info(all_product_links):
    driver.get(all_product_links)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        product_name = soup.find('div', class_ = 'product_detail-title').text.strip()
    except:
        product_name = ''
    try:
        product_url = soup.find('link', {'rel': 'canonical'}).get('href')
    except:
        product_url = ''
    try:
        discounted_price = soup.find('strong', class_ = 'giakm').text.strip()
    except:
        discounted_price = soup.find('span', class_ = 'gia-km-cu').text.strip()
    try:
        original_price = soup.find('strong', class_ = 'giany').text.strip()
    except:
        original_price = discounted_price   
    # installment_payment (Trả góp)
    new_response = requests.get('https://hacom.vn/huong-dan-mua-hang-tra-gop')
    new_soup = BeautifulSoup(new_response.text, 'html.parser')
    installment_payment = new_soup.find('h2', {'id': 'mh-online'}).text.strip()

    max_promotion = 5
    promotion_values =  soup.select_one('.square-trade-content ul')
    # Tạo một từ điển để lưu các giá trị
    promotions = []
    if promotion_values:
    # Lấy danh sách các thông tin khuyến mãi
        for i, li in enumerate(promotion_values.find_all('li'), start=1):
            variable_name = f"Promotion__{i}"
            promotion_text = li.text.strip()
            promotions.append((variable_name, promotion_text))
    
    product_price = {
            'laptop_name': product_name,
            'discounted_price': discounted_price,
            'original_price': original_price,
            'installment_payment' : installment_payment,
            'shop_brand': 'Hacom'
        }
        # Thêm các biến khuyến mại vào dictionary
    for i in range(1, max_promotion + 1):
        key = f'promotion_{i}'
        # Nếu i vượt quá số lượng thực sự của khuyến mại, thiết lập giá trị là None
        if i <= len(promotions):
                value = promotions[i-1][1]
        else:
            value = ''
        product_price[key] = value

    product_price['product_url'] = product_url
    return product_price

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
# Duyệt qua các đường link sản phẩm và lấy thông tin
data = []
for product_link in all_product_links:
    product_price = get_product_info(product_link)
    data.append(product_price)
    print(product_price)

driver.quit()

{'laptop_name': 'Laptop Acer Aspire 5 A514-56P-55K5 (NX.KHRSV.003) (i5 1335U/16GB RAM/512GB SSD/14.0 inch WUXGA IPS/Win11/Xám) (2023)', 'discounted_price': '15.999.000₫', 'original_price': '18.999.000₫', 'installment_payment': 'TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG MUA HÀNG ONLINE', 'shop_brand': 'Hacom', 'promotion_1': 'Balo Acer trị giá 300.000đ (BALO047)', 'promotion_2': 'Tặng phiếu vệ sinh bảo dưỡng Laptop, PC miễn phí trọn đời trị giá 999.000đ (THEK417)', 'promotion_3': '', 'promotion_4': '', 'promotion_5': '', 'product_url': 'https://hacom.vn/laptop-acer-aspire-5-a514-56p-55k5-nx.khrsv.003-i5-1335u-16gb-ram-512gb-ssd-14.0-inch-wuxga-ips-win11-xam-2023'}
{'laptop_name': 'Laptop Lenovo IdeaPad Slim 5 14ILR8 (82XD002VVN) (i5 13500H/16GB RAM/512GB SSD/14 WUXGA OLED/Win11/Xám)', 'discounted_price': '17.499.000₫', 'original_price': '19.999.000₫', 'installment_payment': 'TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG MUA HÀNG ONLINE', 'shop_brand': 'Hacom', 'promotion_1': 'Balo chí

In [29]:
import pandas as pd
df = pd.DataFrame(data)
df

,laptop_name,discounted_price,original_price,installment_payment,shop_brand,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,product_url
0,Laptop Acer Aspire 5 A514-56P-55K5 (NX.KHRSV.0...,15.999.000₫,18.999.000₫,TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG M...,Hacom,Balo Acer trị giá 300.000đ (BALO047),"Tặng phiếu vệ sinh bảo dưỡng Laptop, PC miễn p...",,,,https://hacom.vn/laptop-acer-aspire-5-a514-56p...
1,Laptop Lenovo IdeaPad Slim 5 14ILR8 (82XD002VV...,17.499.000₫,19.999.000₫,TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG M...,Hacom,Balo chính hãng Lenvovo trị giá 299.000đ (BALO...,"Tặng phiếu vệ sinh bảo dưỡng Laptop, PC miễn p...",,,,https://hacom.vn/laptop-lenovo-ideapad-slim-5-...
2,Laptop Dell Inspiron 3520 (71027003) (i5 1235U...,14.299.000₫,16.999.000₫,TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG M...,Hacom,Túi đựng Laptop trị giá 149.000đ (TUID064),"Tặng phiếu vệ sinh bảo dưỡng Laptop, PC miễn p...",,,,https://hacom.vn/laptop-dell-inspiron-3520-710...
3,Laptop Dell Inspiron 3520 (D5N53) (i3 1115G4 8...,9.299.000₫,10.999.000₫,TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG M...,Hacom,Túi đựng Laptop trị giá 149.000đ (TUID064/CAPD...,"Tặng phiếu vệ sinh bảo dưỡng Laptop, PC miễn p...",,,,https://hacom.vn/laptop-dell-inspiron-3520-d5n...
4,Laptop Dell Inspiron 3520 (25P231) (i5 1235U 1...,15.299.000₫,16.999.000₫,TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG M...,Hacom,Túi đựng Laptop trị giá 149.000đ (TUID064/CAPD...,"Tặng phiếu vệ sinh bảo dưỡng Laptop, PC miễn p...",,,,https://hacom.vn/laptop-dell-inspiron-3520-25p...
...,...,...,...,...,...,...,...,...,...,...,...
533,Laptop HP ProBook 440 G6 (5YM73PA) (i7 8565U/8...,19.189.000₫,23.839.000₫,TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG M...,Hacom,,,,,,https://hacom.vn/laptop-hp-probook-440-g6-5ym7...
534,Laptop HP ProBook 450 G8 (2H0W1PA) (i5 1135G7/...,21.999.000₫,22.319.000₫,TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG M...,Hacom,,,,,,https://hacom.vn/laptop-hp-probook-450-g8-2h0w...
535,Laptop HP Pavilion x360 14-dw1019TU (2H3N7PA) ...,23.229.000₫,23.329.000₫,TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG M...,Hacom,,,,,,https://hacom.vn/laptop-hp-pavilion-x360-14-dw...
536,Laptop HP ProBook 440 G6 8AZ16PA (i5 8265U/8GB...,18.079.000₫,18.389.000₫,TRẢ GÓP LÃI SUẤT 0% BẰNG THẺ TÍN DỤNGÁP DỤNG M...,Hacom,,,,,,https://hacom.vn/laptop-hp-probook-440-g6-8az1...


In [30]:
df.to_excel('hacom_price.xlsx', index=False)